<a href="https://colab.research.google.com/github/Tutaseta/DeepDota/blob/master/Option_hedging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
import pandas as pd
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt

import math
from google.colab import files

# Data upload and reading data to Pandas DataFrame

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

data_1 =pd.read_excel('isx2010C.xls', sheet_name='isx15012010C')
data_2 =pd.read_excel('isx2010C.xls', sheet_name='isx19022010C')
data_3 =pd.read_excel('isx2010C.xls', sheet_name='isx17122010C')
data_4 =pd.read_excel('isx2010C.xls', sheet_name='isx19112010C')
data_5 =pd.read_excel('isx2010C.xls', sheet_name='isx15102010C')
data_6 =pd.read_excel('isx2010C.xls', sheet_name='isx17092010C')
data_7 =pd.read_excel('isx2010C.xls', sheet_name='isx20082010C')
data_8 =pd.read_excel('isx2010C.xls', sheet_name='isx16072010C')
data_9 =pd.read_excel('isx2010C.xls', sheet_name='isx18062010C')
data_10=pd.read_excel('isx2010C.xls', sheet_name='isx21052010C')
data_11=pd.read_excel('isx2010C.xls', sheet_name='isx16042010C')
data_12=pd.read_excel('isx2010C.xls', sheet_name='isx19032010C')

Saving isx2010C.xls to isx2010C.xls
User uploaded file "isx2010C.xls" with length 808448 bytes


#Renaming columns

In [0]:
# RENAME COLUMNS ACCORDINGLY
data_1.rename(columns= {'Unnamed: 48':'Spot price','Unnamed: 49':'Risk-free rate'}, inplace=True )
data_2.rename(columns= {'Unnamed: 54':'Spot price','Unnamed: 55':'Risk-free rate'}, inplace=True )
data_3.rename(columns= {'Unnamed: 46':'Spot price','Unnamed: 47':'Risk-free rate'}, inplace=True )
data_4.rename(columns= {'Unnamed: 46':'Spot price','Unnamed: 47':'Risk-free rate'}, inplace=True )
data_5.rename(columns= {'Unnamed: 46':'Spot price','Unnamed: 47':'Risk-free rate'}, inplace=True )
data_6.rename(columns= {'Unnamed: 46':'Spot price','Unnamed: 47':'Risk-free rate'}, inplace=True )
data_7.rename(columns= {'Unnamed: 47':'Spot price','Unnamed: 48':'Risk-free rate'}, inplace=True )
data_8.rename(columns= {'Unnamed: 43':'Spot price','Unnamed: 44':'Risk-free rate'}, inplace=True )
data_9.rename(columns= {'Unnamed: 43':'Spot price','Unnamed: 44':'Risk-free rate'}, inplace=True )
data_10.rename(columns={'Unnamed: 43':'Spot price','Unnamed: 44':'Risk-free rate'}, inplace=True )
data_11.rename(columns={'Unnamed: 45':'Spot price','Unnamed: 46':'Risk-free rate'}, inplace=True )
data_12.rename(columns={'Unnamed: 46':'Spot price','Unnamed: 47':'Risk-free rate'}, inplace=True )

# Black-Scholes, Delta, Vega, Implied Volatility and MSE

In [0]:
def BS(S, K, T, r, sigma):
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    # Returns: Option Price
    return (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))

  
def delta(S, K, T, r, sigma):
  # Rate of change of option value in respect to the price of the underlying asset
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
  return si.norm.cdf(d1, 0.0, 1.0)


def vega(S, K, T, r, sigma):
  # Rate of change of option value in respect to the volatility of underlying asset
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
  return S * si.norm.cdf(d1, 0.0, 1.0) * np.sqrt(T)
  
def imp_vol(call, S, K, T, r, high=2.0, low=0.0001): # Bisection-method
  target = call
  decimals = len(str(target).split('.')[1]) # Count decimals
  for i in range(100000):
    mid = (high + low) / 2
    if mid < 0.00001:
      mid = 0.00001
    estimate = BS(S, K, T, r, mid)
    # IS ESTIMATE CLOSE ENOUGH?
    if round(estimate, decimals) == target: 
      break
    elif estimate > target: 
      high = mid
    elif estimate < target: 
      low = mid
    return mid


def MSE(change):
  return np.mean(np.square(change))

#Delta hedge single option

---
Single options delta hedging

Inputs: **Call price**: dataframe, **spot price**: dataframe, **risk-free**:dataframe, **maturity**: Int, **hedging frequency**: Int


Output: **Hedging error**: float

In [0]:
def delta_hedge(call,spot,rf,strike,maturity,frequency):
  # Days passed
  n=0
  i=0

  A =[] # Changes in values of each portfolio for calculating MSE
  OP=[] # Store options
  RE=[] # Store replicating amount of underlying
  d =[] # Store deltas
  
  # Initial positions
  OP.append(call.iloc[n])
  vol=imp_vol(call.iloc[n], spot.iloc[n], strike, maturity/365, rf.iloc[n])
  d.append(delta(spot.iloc[n], strike, maturity/365, rf.iloc[n], vol))
  RE.append(d[i]*spot.iloc[n])

  # MAIN LOOP
  while n+1<maturity:
    n+=1 #Day passes
    OP.append(call.iloc[n])
    RE.append(d[i]*spot.iloc[n])
    A.append((OP[n]-OP[n-1])-(RE[n]-RE[n-1]))
    
    ### REHEDGE POSITION ###
    if n>2 and n%frequency==0: # Update hedge position
      i+=1
      # Update volatility
      vol=imp_vol(call.iloc[n], spot.iloc[n], strike, (maturity-n)/365, rf.iloc[n])
      
      # Update delta
      d.append(delta(spot.iloc[n], strike, (maturity-n)/365, rf.iloc[n], vol))
      
  # RETURN: Hedging error
  return MSE(A)

In [0]:
# SHEET 1
spot1=data_1['Spot price']
spot1=spot1.drop(spot1.index[0:42]).dropna()

rf1=data_1['Risk-free rate']
rf1=rf1.drop(rf1.index[0:42]).dropna()

atm1=data_1.iloc[:,37] # ATM call Maturity = 45days, Strike = 520, Spot = 516.54
atm1=atm1.drop(atm1.index[0:42]).dropna()

itm1=data_1.iloc[:,31] # ITM  Strike = 490
itm1=itm1.drop(itm1.index[0:42]).dropna()

otm1=data_1.iloc[:,43] # OTM Strike = 550
otm1=otm1.drop(otm1.index[0:42]).dropna()

maturity=43.0

atm_strike=520.0
itm_strike=490.0
otm_strike=550.0

print("Sheet 1 ATM, Hedging error, everyother day: ",delta_hedge(atm1,spot1,rf1,atm_strike,maturity,2)) # everyother day
print("Sheet 1 ATM, Hedging error, every 7 days:   ",delta_hedge(atm1,spot1,rf1,atm_strike,maturity,7)) # once a week

print("Sheet 1 ITM, Hedging error, everyother day: ",delta_hedge(itm1,spot1,rf1,itm_strike,maturity,2)) # everyother day
print("Sheet 1 ITM, Hedging error, every 7 days:   ",delta_hedge(itm1,spot1,rf1,itm_strike,maturity,7)) # once a week

print("Sheet 1 OTM, Hedging error, everyother day: ",delta_hedge(otm1,spot1,rf1,otm_strike,maturity,2)) # everyother day
print("Sheet 1 OTM, Hedging error, every 7 days:   ",delta_hedge(otm1,spot1,rf1,otm_strike,maturity,7)) # once a week

Sheet 1 ATM, Hedging error, everyother day:  28.0109379000415
Sheet 1 ATM, Hedging error, every 7 days:    17.998682378374173
Sheet 1 ITM, Hedging error, everyother day:  60.22800058967588
Sheet 1 ITM, Hedging error, every 7 days:    38.44415038142632
Sheet 1 OTM, Hedging error, everyother day:  292403.93350430153
Sheet 1 OTM, Hedging error, every 7 days:    291602.32492777286


In [0]:
# SHEET 2
spot2=data_2['Spot price']
spot2=spot2.drop(spot2.index[0:45]).dropna()

rf2=data_2['Risk-free rate']
rf2=rf2.drop(rf2.index[0:45]).dropna()

atm2=data_2.iloc[:,37] # ATM call Maturity = 45days, Strike = 515, Spot = 513.94
atm2=atm2.drop(atm2.index[0:45]).dropna()

itm2=data_2.iloc[:,31] # ITM  Strike = 485
itm2=itm2.drop(itm2.index[0:45]).dropna()

otm2=data_2.iloc[:,43] # OTM Strike = 545
otm2=otm2.drop(otm2.index[0:45]).dropna()

atm_strike2=515
itm_strike2=485
otm_strike2=545

print("Sheet 2 ATM, Hedging error, everyother day: ",delta_hedge(atm2,spot2,rf2,atm_strike2,maturity,2)) # everyother day
print("Sheet 2 ATM, Hedging error, every 7 days:   ",delta_hedge(atm2,spot2,rf2,atm_strike2,maturity,7)) # once a week

print("Sheet 2 ITM, Hedging error, everyother day: ",delta_hedge(itm2,spot2,rf2,itm_strike2,maturity,2)) # everyother day
print("Sheet 2 ITM, Hedging error, every 7 days:   ",delta_hedge(itm2,spot2,rf2,itm_strike2,maturity,7)) # once a week

print("Sheet 2 OTM, Hedging error, everyother day: ",delta_hedge(otm2,spot2,rf2,otm_strike2,maturity,2)) # everyother day
print("Sheet 2 OTM, Hedging error, every 7 days:   ",delta_hedge(otm2,spot2,rf2,otm_strike2,maturity,7)) # once a week

Sheet 2 ATM, Hedging error, everyother day:  ([-0.49111424938267945, -0.285638952331448, -0.3335791391081955, 0.0, -5.676990393501024, 0.22086731691405248, -0.38163286745738034, 1.064273345830447, 6.744542125123701, -1.7258838380994987, -11.468545556512026, -0.1293056484908437, -1.4374489208074923, -0.13943585822987714, -3.5375604738766278, 0.7575181136245916, 4.789661489613252, 0.2767375722243628, -7.95709515841412, 0.0, 8.287405224400265, -0.10870209348699866, 0.9636388627651051, 4.669504096036337, 35.99673616763423, 0.3142292598014138, 1.0899210591171427, 1.6044419473683944, 10.31130320180572, -2.8815675078205114, -2.4659675706900615, 0.8568073633400219, 1.595988070763657, -0.8290095314390953, 39.418010329924435, -2.591629554383843, 5.354599350313748, -1.8022118567697476, 2.4495672859322815, 0.0, 23.968295859661705, -0.6730804974526166], 95.34723534966868)
Sheet 2 ATM, Hedging error, every 7 days:    ([-0.49111424938267945, -0.285638952331448, -0.3335791391081955, 0.0, -0.4540337371

In [0]:
# SHEET 3
spot3=data_3['Spot price']
spot3=spot.drop(spot.index[0:133]).dropna()

rf3=data_3['Risk-free rate']
rf3=rf3.drop(rf3.index[0:133]).dropna()

atm3=data_3.iloc[:,40] # ATM call Maturity = 45days, Strike = 535, Spot = 535.23
atm3=atm.drop(atm.index[0:133]).dropna()

itm3=data_3.iloc[:,37]
itm3=itm3.drop(itm3.index[0:45]).dropna()

otm2=data_2.iloc[:,43]
otm2=otm2.drop(otm2.index[0:45]).dropna()

atm_strike3=535
itm_strike3=520
otm_strike3=550


print("Sheet 3 ATM, Hedging error, everyother day: ",delta_hedge(atm3,spot3,rf3,atm_strike3,maturity,2)) # everyother day
print("Sheet 3 ATM, Hedging error, every 7 days:   ",delta_hedge(atm3,spot3,rf3,atm_strike3,maturity,7)) # once a week

print("Sheet 3 ITM, Hedging error, everyother day: ",delta_hedge(itm3,spot3,rf3,itm_strike3,maturity,2)) # everyother day
print("Sheet 3 ITM, Hedging error, every 7 days:   ",delta_hedge(itm3,spot3,rf3,itm_strike3,maturity,7)) # once a week

print("Sheet 3 OTM, Hedging error, everyother day: ",delta_hedge(otm3,spot3,rf3,otm_strike3,maturity,2)) # everyother day
print("Sheet 3 OTM, Hedging error, every 7 days:   ",delta_hedge(otm3,spot3,rf3,otm_strike3,maturity,7)) # once a week

Sheet 3, Hedging error, everyother day:  ([1.9062588071400892, -0.6373421923618068, -1.2791738161554864, -0.32882047626818434, 4.206270043016115, -0.8247130638245395, -0.5534329725531837, 0.6463654242651611, 2.5278843062554195, -0.04415594562371972, 1.694328431491181, -1.4280908599804096, -11.197098322159844, 3.255185756515008, -15.249466593435674, -0.050490817760316276, 6.778359520694906, -0.6121121950862545, 2.308633346286854, 0.5582310098857981, 11.530912116444632, -0.1484652977755445, 13.98929216163566, -1.219745892259267, -14.611193503828735, 1.0169082764626411, 17.140535936623415, 0.0, -11.379480051371331, 0.18988542229406047, 10.420115981674064, -0.8327326549665832, -14.604530552202672, -0.623334280956616, -18.459407235239475, 0.6156973858754444, -2.58676818101128, 1.1521191741416956, -20.591421123755232, 0.7511733514258268, -28.97500553106269, 0.5120754005274826], 80.02786738590139)
Sheet 3, Hedging error, every 7 days:    ([1.9062588071400892, -0.6373421923618068, -1.279173816

In [0]:
# SHEET 4
spot4=data_4['Spot price']
spot4=spot.drop(spot4.index[0:45]).dropna()

rf4=data_4['Risk-free rate']
rf4=rf4.drop(rf4.index[0:133]).dropna()

atm4=data_4.iloc[:,36] # ATM call Maturity = 45days, Strike = 535, Spot = 535.23
atm4=atm4.drop(atm4.index[0:133]).dropna()

itm4=data_4.iloc[:,31]
itm4=itm4.drop(itm4.index[0:45]).dropna()

otm4=data_4.iloc[:,41]
otm4=otm4.drop(otm4.index[0:45]).dropna()

atm_strike4=515
itm_strike4=490
otm_strike4=550


print("Sheet 4 ATM, Hedging error, everyother day: ",delta_hedge(atm4,spot4,rf4,atm_strike4,maturity,2)) # everyother day
print("Sheet 4 ATM, Hedging error, every 7 days:   ",delta_hedge(atm4,spot4,rf4,atm_strike4,maturity,7)) # once a week

print("Sheet 4 ITM, Hedging error, everyother day: ",delta_hedge(itm4,spot4,rf4,itm_strike4,maturity,2)) # everyother day
print("Sheet 4 ITM, Hedging error, every 7 days:   ",delta_hedge(itm4,spot4,rf4,itm_strike4,maturity,7)) # once a week

print("Sheet 4 OTM, Hedging error, everyother day: ",delta_hedge(otm4,spot4,rf4,otm_strike4,maturity,2)) # everyother day
print("Sheet 4 OTM, Hedging error, every 7 days:   ",delta_hedge(otm4,spot4,rf4,otm_strike4,maturity,7)) # once a week

Sheet 4, Hedging error, everyother day:  ([0.24074669802996596, -0.09485616066164049, 0.8338109281817196, -1.6416594898021124, -1.7442944202860424, -0.40498351097909335, 3.211645135464643, -0.12002043200644863, 4.776301398850933, -0.331751848049441, 1.7067805530058546, 0.6790118976158155, -14.54894654416297, 0.7703063687431495, -0.17045621252752952, -2.772925574812067, 0.04658520516973397, -1.8464783362998851, -1.854513548289237, 0.6233517610280188, -7.817277396014639, 1.0968176228206516, 2.8929919135113416, 0.701056176695058, -2.683402512117791, -1.053965018282156, -3.213801012260479, 0.6627306436120897, 1.643456403877508, 0.5209999813791342, 0.226054366270386, 1.9906489064172987, -16.02089008542731, 0.9924758397648219, -31.21747794929105, 1.972898256319489, -1.5981686894968945, 0.848242748106248, -10.890540939403195, 0.3814276433580055, 2.811311436950529, -2.4729948038322256], 41.42551157475967)
Sheet 4, Hedging error, every 7 days:    ([0.24074669802996596, -0.09485616066164049, 0.8

In [0]:
# SHEET 5
spot5=data_5['Spot price']
spot5=spot5.drop(spot5.index[0:45]).dropna()

rf5=data_5['Risk-free rate']
rf5=rf5.drop(rf5.index[0:133]).dropna()

atm5=data_5.iloc[:,30] 
atm5=atm5.drop(atm5.index[0:133]).dropna()

itm5=data_5.iloc[:,27]
itm5=itm5.drop(itm5.index[0:45]).dropna()

otm5=data_5.iloc[:,33]
otm5=otm5.drop(otm5.index[0:45]).dropna()

atm_strike5=485
itm_strike5=470
otm_strike5=500


print("Sheet 5 ATM, Hedging error, everyother day: ",delta_hedge(atm5,spot5,rf5,atm_strike5,maturity,2)) # everyother day
print("Sheet 5 ATM, Hedging error, every 7 days:   ",delta_hedge(atm5,spot5,rf5,atm_strike5,maturity,7)) # once a week

print("Sheet 5 ITM, Hedging error, everyother day: ",delta_hedge(itm5,spot5,rf5,itm_strike5,maturity,2)) # everyother day
print("Sheet 5 ITM, Hedging error, every 7 days:   ",delta_hedge(itm5,spot5,rf5,itm_strike5,maturity,7)) # once a week

print("Sheet 5 OTM, Hedging error, everyother day: ",delta_hedge(otm5,spot5,rf5,otm_strike5,maturity,2)) # everyother day
print("Sheet 5 OTM, Hedging error, every 7 days:   ",delta_hedge(otm5,spot5,rf5,otm_strike5,maturity,7)) # once a week

In [0]:
# SHEET 6
spot6=data_6['Spot price']
spot6=spot6.drop(spot6.index[0:45]).dropna()

rf6=data_6['Risk-free rate']
rf6=rf6.drop(rf6.index[0:133]).dropna()

atm6=data_6.iloc[:,30] 
atm6=atm6.drop(atm6.index[0:133]).dropna()

itm6=data_6.iloc[:,27]
itm6=itm6.drop(itm6.index[0:45]).dropna()

otm6=data_6.iloc[:,33]
otm6=otm6.drop(otm6.index[0:45]).dropna()

atm_strike6=485
itm_strike6=470
otm_strike6=500


print("Sheet 6 ATM, Hedging error, everyother day: ",delta_hedge(atm6,spot6,rf6,atm_strike6,maturity,2)) # everyother day
print("Sheet 6 ATM, Hedging error, every 7 days:   ",delta_hedge(atm6,spot6,rf6,atm_strike6,maturity,7)) # once a week

print("Sheet 6 ITM, Hedging error, everyother day: ",delta_hedge(itm6,spot6,rf6,itm_strike6,maturity,2)) # everyother day
print("Sheet 6 ITM, Hedging error, every 7 days:   ",delta_hedge(itm6,spot6,rf6,itm_strike6,maturity,7)) # once a week

print("Sheet 6 OTM, Hedging error, everyother day: ",delta_hedge(otm6,spot6,rf6,otm_strike6,maturity,2)) # everyother day
print("Sheet 6 OTM, Hedging error, every 7 days:   ",delta_hedge(otm6,spot6,rf6,otm_strike6,maturity,7)) # once a week

In [0]:
# SHEET 7
spot7=data_7['Spot price']
spot7=spot7.drop(spot7.index[0:45]).dropna()

rf7=data_7['Risk-free rate']
rf7=rf7.drop(rf7.index[0:133]).dropna()

atm7=data_7.iloc[:,34] 
atm7=atm7.drop(atm7.index[0:133]).dropna()

itm7=data_7.iloc[:,31]
itm7=itm7.drop(itm7.index[0:45]).dropna()

otm7=data_7.iloc[:,37]
otm7=otm7.drop(otm7.index[0:45]).dropna()

atm_strike7=505
itm_strike7=490
otm_strike7=520


print("Sheet 7 ATM, Hedging error, everyother day: ",delta_hedge(atm7,spot7,rf7,atm_strike7,maturity,2)) # everyother day
print("Sheet 7 ATM, Hedging error, every 7 days:   ",delta_hedge(atm7,spot7,rf7,atm_strike7,maturity,7)) # once a week

print("Sheet 7 ITM, Hedging error, everyother day: ",delta_hedge(itm7,spot7,rf7,itm_strike7,maturity,2)) # everyother day
print("Sheet 7 ITM, Hedging error, every 7 days:   ",delta_hedge(itm7,spot7,rf7,itm_strike7,maturity,7)) # once a week

print("Sheet 7 OTM, Hedging error, everyother day: ",delta_hedge(otm7,spot7,rf7,otm_strike7,maturity,2)) # everyother day
print("Sheet 7 OTM, Hedging error, every 7 days:   ",delta_hedge(otm7,spot7,rf7,otm_strike7,maturity,7)) # once a week

In [0]:
# SHEET 8
spot8=data_8['Spot price']
spot8=spot8.drop(spot8.index[0:45]).dropna()

rf8=data_8['Risk-free rate']
rf8=rf8.drop(rf8.index[0:133]).dropna()

atm8=data_8.iloc[:,34] 
atm8=atm8.drop(atm8.index[0:133]).dropna()

itm8=data_8.iloc[:,30]
itm8=itm8.drop(itm8.index[0:45]).dropna()

otm8=data_8.iloc[:,38]
otm8=otm8.drop(otm8.index[0:45]).dropna()

atm_strike8=520
itm_strike8=500
otm_strike8=540


print("Sheet 8 ATM, Hedging error, everyother day: ",delta_hedge(atm8,spot8,rf8,atm_strike8,maturity,2)) # everyother day
print("Sheet 8 ATM, Hedging error, every 7 days:   ",delta_hedge(atm8,spot8,rf8,atm_strike8,maturity,7)) # once a week

print("Sheet 8 ITM, Hedging error, everyother day: ",delta_hedge(itm8,spot8,rf8,itm_strike8,maturity,2)) # everyother day
print("Sheet 8 ITM, Hedging error, every 7 days:   ",delta_hedge(itm8,spot8,rf8,itm_strike8,maturity,7)) # once a week

print("Sheet 8 OTM, Hedging error, everyother day: ",delta_hedge(otm8,spot8,rf8,otm_strike8,maturity,2)) # everyother day
print("Sheet 8 OTM, Hedging error, every 7 days:   ",delta_hedge(otm8,spot8,rf8,otm_strike8,maturity,7)) # once a week

In [0]:
# SHEET 9
spot9=data_9['Spot price']
spot9=spot9.drop(spot9.index[0:45]).dropna()

rf9=data_9['Risk-free rate']
rf9=rf9.drop(rf9.index[0:133]).dropna()

atm9=data_9.iloc[:,40] 
atm9=atm9.drop(atm9.index[0:133]).dropna()

itm9=data_9.iloc[:,38]
itm9=itm9.drop(itm9.index[0:45]).dropna()

otm9=data_9.iloc[:,42]
otm9=otm9.drop(otm9.index[0:45]).dropna()

atm_strike9=550
itm_strike9=540
otm_strike9=560


print("Sheet 9 ATM, Hedging error, everyother day: ",delta_hedge(atm9,spot9,rf9,atm_strike9,maturity,2)) # everyother day
print("Sheet 9 ATM, Hedging error, every 7 days:   ",delta_hedge(atm9,spot9,rf9,atm_strike9,maturity,7)) # once a week

print("Sheet 9 ITM, Hedging error, everyother day: ",delta_hedge(itm9,spot9,rf9,itm_strike9,maturity,2)) # everyother day
print("Sheet 9 ITM, Hedging error, every 7 days:   ",delta_hedge(itm9,spot9,rf9,itm_strike9,maturity,7)) # once a week

print("Sheet 9 OTM, Hedging error, everyother day: ",delta_hedge(otm9,spot9,rf9,otm_strike9,maturity,2)) # everyother day
print("Sheet 9 OTM, Hedging error, every 7 days:   ",delta_hedge(otm9,spot9,rf9,otm_strike9,maturity,7)) # once a week

In [0]:
# SHEET 10
spot10=data_10['Spot price']
spot10=spot10.drop(spot10.index[0:45]).dropna()

rf10=data_10['Risk-free rate']
rf10=rf10.drop(rf10.index[0:133]).dropna()

atm10=data_10.iloc[:,37] 
atm10=atm10.drop(atm10.index[0:133]).dropna()

itm10=data_10.iloc[:,34]
itm10=itm10.drop(itm10.index[0:45]).dropna()

otm10=data_10.iloc[:,40]
otm10=otm10.drop(otm10.index[0:45]).dropna()

atm_strike10=535
itm_strike10=520
otm_strike10=550


print("Sheet 10 ATM, Hedging error, everyother day: ",delta_hedge(atm10,spot10,rf10,atm_strike10,maturity,2)) # everyother day
print("Sheet 10 ATM, Hedging error, every 7 days:   ",delta_hedge(atm10,spot10,rf10,atm_strike10,maturity,7)) # once a week

print("Sheet 10 ITM, Hedging error, everyother day: ",delta_hedge(itm10,spot10,rf10,itm_strike10,maturity,2)) # everyother day
print("Sheet 10 ITM, Hedging error, every 7 days:   ",delta_hedge(itm10,spot10,rf10,itm_strike10,maturity,7)) # once a week

print("Sheet 10 OTM, Hedging error, everyother day: ",delta_hedge(otm10,spot10,rf10,otm_strike10,maturity,2)) # everyother day
print("Sheet 10 OTM, Hedging error, every 7 days:   ",delta_hedge(otm10,spot10,rf10,otm_strike10,maturity,7)) # once a week

In [0]:
# SHEET 11
spot11=data_11['Spot price']
spot11=spot11.drop(spot11.index[0:45]).dropna()

rf11=data_11['Risk-free rate']
rf11=rf11.drop(rf11.index[0:133]).dropna()

atm11=data_11.iloc[:,32] 
atm11=atm11.drop(atm11.index[0:133]).dropna()

itm11=data_11.iloc[:,28]
itm11=itm11.drop(itm11.index[0:45]).dropna()

otm11=data_11.iloc[:,34]
otm11=otm11.drop(otm11.index[0:45]).dropna()

atm_strike11=500
itm_strike11=480
otm_strike11=520


print("Sheet 11 ATM, Hedging error, everyother day: ",delta_hedge(atm11,spot11,rf11,atm_strike11,maturity,2)) # everyother day
print("Sheet 11 ATM, Hedging error, every 7 days:   ",delta_hedge(atm11,spot11,rf11,atm_strike11,maturity,7)) # once a week

print("Sheet 11 ITM, Hedging error, everyother day: ",delta_hedge(itm11,spot11,rf11,itm_strike11,maturity,2)) # everyother day
print("Sheet 11 ITM, Hedging error, every 7 days:   ",delta_hedge(itm11,spot11,rf11,itm_strike11,maturity,7)) # once a week

print("Sheet 11 OTM, Hedging error, everyother day: ",delta_hedge(otm11,spot11,rf11,otm_strike11,maturity,2)) # everyother day
print("Sheet 11 OTM, Hedging error, every 7 days:   ",delta_hedge(otm11,spot11,rf11,otm_strike11,maturity,7)) # once a week

In [0]:
# SHEET 12
spot12=data_12['Spot price']
spot12=spot12.drop(spot12.index[0:45]).dropna()

rf12=data_12['Risk-free rate']
rf12=rf12.drop(rf12.index[0:133]).dropna()

atm12=data_12.iloc[:,37] 
atm12=atm12.drop(atm12.index[0:133]).dropna()

itm12=data_12.iloc[:,33]
itm12=itm12.drop(itm12.index[0:45]).dropna()

otm12=data_12.iloc[:,41]
otm12=otm12.drop(otm12.index[0:45]).dropna()

atm_strike12=520
itm_strike12=500
otm_strike12=540


print("Sheet 12 ATM, Hedging error, everyother day: ",delta_hedge(atm12,spot12,rf12,atm_strike12,maturity,2)) # everyother day
print("Sheet 12 ATM, Hedging error, every 7 days:   ",delta_hedge(atm12,spot12,rf12,atm_strike12,maturity,7)) # once a week

print("Sheet 12 ITM, Hedging error, everyother day: ",delta_hedge(itm12,spot12,rf12,itm_strike12,maturity,2)) # everyother day
print("Sheet 12 ITM, Hedging error, every 7 days:   ",delta_hedge(itm12,spot12,rf12,itm_strike12,maturity,7)) # once a week

print("Sheet 12 OTM, Hedging error, everyother day: ",delta_hedge(otm12,spot12,rf12,otm_strike12,maturity,2)) # everyother day
print("Sheet 12 OTM, Hedging error, every 7 days:   ",delta_hedge(otm12,spot12,rf12,otm_strike12,maturity,7)) # once a week

#Delta-Vega hedge single option
---
Short position with maturity T

Alpha-amount of underlying stock

Eta-amount of replicating option, same underlying with same strike prices, but a longer maturity T2>T

---
Take replicating call from another sheet.

![](https://drive.google.com/uc?export=view&id=1OF4JhVxUrpjZpnP1hWH_GxYQylSEjgG1) Amount of stock

![](https://drive.google.com/uc?export=view&id=1waEtPWmJAbcP-m3kOm4x_u5R8XSKwl3U) Amount of replicating option

![](https://drive.google.com/uc?export=view&id=1Xs1FB9GAsnewOJhJNU4F37MCXncKdada)

In [0]:
def delta_vega_hedge(call,spot,rf,strike,maturity,frequency,rep_call,rep_spot,rep_rf,rep_maturity):
  n=0 # Days passed
  i=0 # Used for rehedging
  
  # Difference between change in the value of the option
  # and the change in the value of replicating portfolio
  A    =[] # Store hedging error
  OP   =[] # Store hedged option
  stock=[] # Store alpha amounts of underlying
  rep  =[] # Store eta amounts of replicating stock
  alpha=[] # Store alphas
  eta  =[] # Store etas
  
  ### Initial positions ###
  
  # Hedged option
  OP.append(call.iloc[n])
  
  # Implied volatilites for hedged option and replicating option
  vol_BS =imp_vol(call.iloc[n], spot.iloc[n], strike, maturity/365, rf.iloc[n])
  vol_rep=imp_vol(rep_call.iloc[n],rep_spot.iloc[n],strike,rep_maturity/365,rep_rf.iloc[n])
  
  # Deltas for positions
  d    =-delta(spot.iloc[n], strike, maturity/365, rf.iloc[n], vol_BS)
  d_rep=delta(rep_spot.iloc[n],strike,rep_maturity/365,rep_rf.iloc[n],vol_rep)
  
  # Vegas for positions
  v    =vega(spot.iloc[n], strike, maturity/365, rf.iloc[n], vol_BS)
  v_rep=vega(rep_spot.iloc[n],strike,rep_maturity/365,rep_rf.iloc[n],vol_rep)
  
  # Alpha amount of underlying stock
  alpha.append(d+(v/v_rep)*d_rep)
  stock.append(alpha[i]*spot.iloc[n])
  
  # Eta amount of replicating option
  eta.append(-v/v_rep) 
  rep.append(eta[i]*rep_call.iloc[n])
  
  # MAIN LOOP
  while n+1<maturity:
    n+=1 # Day passes
    
    ### Calculate new values of positions ###
    
    # Use i as index for alpha and eta
    OP.append(call.iloc[n])
    stock.append(alpha[i]*spot.iloc[n])
    rep.append(eta[i]*rep_call.iloc[n])
   
    A.append((OP[n]-OP[n-1])-(stock[n]-stock[n-1])-(rep[n]-rep[n-1]))
    
    ### REHEDGE POSITIONS ###
    if n>2 and n%frequency==0:
      i+=1
      # Update volatilities for hedged option and replicating option
      vol_BS =imp_vol(call.iloc[n], spot.iloc[n], strike, maturity/365, rf.iloc[n])
      vol_rep=imp_vol(rep_call.iloc[n],rep_spot.iloc[n],strike,rep_maturity/365,rep_rf.iloc[n])
      
      # Update deltas 
      d    =-delta(spot.iloc[n], strike, maturity/365, rf.iloc[n], vol_BS)
      d_rep=delta(rep_spot.iloc[n],strike,rep_maturity/365,rep_rf.iloc[n],vol_rep)
      
      # Update vegas
      v    =vega(spot.iloc[n], strike, maturity/365, rf.iloc[n], vol_BS)
      v_rep=vega(rep_spot.iloc[n],strike,rep_maturity/365,rep_rf.iloc[n],vol_rep)
      
      # Update hedging amounts accordingly
      alpha.append(d+(v/v_rep)*d_rep)
      eta.append(-v/v_rep)
      
      # Rehedge positions at the end of the day?
      
  return MSE(A)

In [0]:
# Hedging option from SHEET 1 and Replicating option from SHEET 2

# SHEET 1
spot1=data_1['Spot price']
spot1=spot1.drop(spot1.index[0:42]).dropna()

rf1=data_1['Risk-free rate']
rf1=rf1.drop(rf1.index[0:42]).dropna()

atm1=data_1.iloc[:,37] # ATM call Maturity = 45days, Strike = 520, Spot = 516.54
atm1=atm1.drop(atm1.index[0:42]).dropna()

itm1=data_1.iloc[:,31] # ITM  Strike = 490
itm1=itm1.drop(itm1.index[0:42]).dropna()

otm1=data_1.iloc[:,43] # OTM Strike = 550
otm1=otm1.drop(otm1.index[0:42]).dropna()

maturity1=43.0
strike1=520.0

# SHEET 2

# Replicating option
rep_atm=data_2.iloc[:,37]
rep_atm=rep_atm.drop(rep_atm.index[0:25]).dropna()

rep_itm=data_2.iloc[:,31]
rep_itm=rep_itm.drop(rep_itm.index[0:25]).dropna()
strike_itm=490.0

rep_otm=data_2.iloc[:,43]
rep_otm=rep_otm.drop(rep_otm.index[0:25]).dropna()
strike_otm=550.0

maturity2=63.0

# Replicating spot price
rep_spot=data_2['Spot price']
rep_spot=rep_spot.drop(rep_spot.index[0:25]).dropna()

# Replicating option interest-rate
rep_rf=data_2['Risk-free rate']
rep_rf=rep_rf.drop(rep_rf.index[0:25]).dropna()

print("Sheet 1 hedged option ATM, sheet 2 replicating option, Hedging error, everyother day: ",delta_vega_hedge(atm1,spot1,rf1,strike1,maturity1,2,rep_atm,rep_spot,rep_rf,maturity2))
print("Sheet 1 hedged option ATM, sheet 2 replicating option, Hedging error, every 7 days:   ",delta_vega_hedge(atm1,spot1,rf1,strike1,maturity1,7,rep_atm,rep_spot,rep_rf,maturity2))

print("Sheet 1 hedged option ITM, sheet 2 replicating option, Hedging error, everyother day: ",delta_vega_hedge(itm1,spot1,rf1,strike_itm,maturity1,2,rep_itm,rep_spot,rep_rf,maturity2))
print("Sheet 1 hedged option ITM, sheet 2 replicating option, Hedging error, every 7 days:   ",delta_vega_hedge(itm1,spot1,rf1,strike_itm,maturity1,7,rep_itm,rep_spot,rep_rf,maturity2))

print("Sheet 1 hedged option OTM, sheet 2 replicating option, Hedging error, everyother day: ",delta_vega_hedge(otm1,spot1,rf1,strike_otm,maturity1,2,rep_otm,rep_spot,rep_rf,maturity2))
print("Sheet 1 hedged option OTM, sheet 2 replicating option, Hedging error, every 7 days:   ",delta_vega_hedge(otm1,spot1,rf1,strike_otm,maturity1,7,rep_otm,rep_spot,rep_rf,maturity2))

Sheet 1 hedged option ATM, sheet 2 replicating option, Hedging error, everyother day:  ([-4.574985363899323, -0.24587820417262307, 2.8107933742583366, -2.767677652759783, -2.583477676821232, 2.079378154929426, -1.559378153951922, -0.2726503856098752, 3.0615230646719027, -2.661484844355389, -4.176372775778214, 1.0877635154189278, -0.7876649478408844, -2.0126814739807157, 3.397959001459827, -0.1217367797743254, -0.8045549238233374, -2.2778294350644837, -5.6232973035971225, 0.48228320639967315, 1.8178400973965498, 0.5015638783430205, 3.3867893040698176, 0.2724818066260548, -0.1914618096996712, 1.473460710142203, -0.7370288056462098, -1.2782573234080807, -2.134382007435063, 2.484267638852918, 1.8756601616000017, -0.23580403763124913, 6.38280838126448, 1.2331684115931356, -0.4030794551152299, -0.20926476912342995, 0.028285878978172363, 0.6105963955801261, -4.305043041929701, 2.579724196636268, 1.1612123841318205, -9.27207643359813], 7.814168849881856)
Sheet 1 hedged option ATM, sheet 2 repl

In [0]:
# Hedging option from SHEET 2 and Replicating option from SHEET 3

# SHEET 2
spot2=data_2['Spot price']
spot2=spot2.drop(spot2.index[0:42]).dropna()

rf2=data_2['Risk-free rate']
rf2=rf2.drop(rf1.index[0:42]).dropna()

atm2=data_2.iloc[:,37] # ATM call Maturity = 45days, Strike = 520, Spot = 516.54
atm2=atm2.drop(atm2.index[0:42]).dropna()

itm1=data_1.iloc[:,31] # ITM  Strike = 490
itm1=itm1.drop(itm1.index[0:42]).dropna()

otm1=data_1.iloc[:,43] # OTM Strike = 550
otm1=otm1.drop(otm1.index[0:42]).dropna()

maturity1=43.0
strike1=520.0

# SHEET 3

# Replicating option
rep_atm=data_2.iloc[:,37]
rep_atm=rep_atm.drop(rep_atm.index[0:25]).dropna()

rep_itm=data_2.iloc[:,31]
rep_itm=rep_itm.drop(rep_itm.index[0:25]).dropna()
strike_itm=490.0

rep_otm=data_2.iloc[:,43]
rep_otm=rep_otm.drop(rep_otm.index[0:25]).dropna()
strike_otm=550.0

maturity2=63.0

# Replicating spot price
rep_spot=data_2['Spot price']
rep_spot=rep_spot.drop(rep_spot.index[0:25]).dropna()

# Replicating option interest-rate
rep_rf=data_2['Risk-free rate']
rep_rf=rep_rf.drop(rep_rf.index[0:25]).dropna()

print("Sheet 2 hedged option ATM, sheet 3 replicating option, Hedging error, everyother day: ",delta_vega_hedge(atm1,spot1,rf1,strike1,maturity1,2,rep_atm,rep_spot,rep_rf,maturity2))
print("Sheet 2 hedged option ATM, sheet 3 replicating option, Hedging error, every 7 days:   ",delta_vega_hedge(atm1,spot1,rf1,strike1,maturity1,7,rep_atm,rep_spot,rep_rf,maturity2))

print("Sheet 2 hedged option ITM, sheet 3 replicating option, Hedging error, everyother day: ",delta_vega_hedge(itm1,spot1,rf1,strike_itm,maturity1,2,rep_itm,rep_spot,rep_rf,maturity2))
print("Sheet 2 hedged option ITM, sheet 3 replicating option, Hedging error, every 7 days:   ",delta_vega_hedge(itm1,spot1,rf1,strike_itm,maturity1,7,rep_itm,rep_spot,rep_rf,maturity2))

print("Sheet 2 hedged option OTM, sheet 3 replicating option, Hedging error, everyother day: ",delta_vega_hedge(otm1,spot1,rf1,strike_otm,maturity1,2,rep_otm,rep_spot,rep_rf,maturity2))
print("Sheet 2 hedged option OTM, sheet 3 replicating option, Hedging error, every 7 days:   ",delta_vega_hedgea(otm1,spot1,rf1,strike_otm,maturity1,7,rep_otm,rep_spot,rep_rf,maturity2))

NameError: ignored

#Delta hedge porfolio of options
---
Same maturities

In [0]:
def delta_hedge_portfolio(call,spot,rf,strike,maturity,frequency,call1,spot1,rf1): # Same maturity for portfolio both options
  # Days passed
  n=0 
  i=0
  
  A  =[]
  
  # First option
  OP =[]
  RE =[]
  d  =[]
  
  # Second option
  OP1=[]
  RE1=[]
  d1 =[]
  
  ### INITIAL POSITIONS ###
  
  # Initial position for FIRST option and replicating delta amount of underlying
  OP.append(call.iloc[n])
  vol=imp_vol(call.iloc[n], spot.iloc[n], strike, (maturity-n)/365, rf.iloc[n])
  d.append(delta(spot.iloc[n], strike, (maturity-n)/365, rf.iloc[n], vol))
  RE.append(d[i]*spot.iloc[n])
  
  # Initial position for SECOND option and replicating delta amount of underlying
  OP1.append(call1.iloc[n])
  vol1=imp_vol(call1.iloc[n], spot1.iloc[n], strike, (maturity-n)/365, rf1.iloc[n])
  d1.append(delta(spot1.iloc[n], strike, (maturity-n)/365, rf1.iloc[n], vol1))
  RE1.append(d[i]*spot1.iloc[n])

  # MAIN LOOP
  while n+1<maturity:
    # Day passes
    n+=1
    
    OP.append(call.iloc[n])
    RE.append(d[i]*spot.iloc[n])
    
    OP1.append(call1.iloc[n])
    RE1.append(d1[i]*spot1.iloc[n])
    
    A.append((OP[n]-OP[n-1])+(OP1[n]-OP1[n-1])-(RE[n]-RE[n-1])-(RE1[n]-RE1[n-1]))

    ### REHEDGE POSITION ###
    if n>2 and n%frequency==0:
      i+=1
      # First option
      vol=imp_vol(call.iloc[n], spot.iloc[n], strike, (maturity-n)/365, rf.iloc[n])
      d.append(delta(spot.iloc[n], strike, (maturity-n)/365, rf.iloc[n], vol))
      
      # Second option
      vol1=imp_vol(call1.iloc[n], spot1.iloc[n], strike, maturity/365, rf1.iloc[n])
      d1.append(delta(spot1.iloc[n], strike, (maturity-n)/365, rf1.iloc[n],vol1))
      
  # RETURN: Hedging error
  return MSE(A)

In [0]:
# SHEET 1
spot1=data_1['Spot price']
spot1=spot1.drop(spot1.index[0:42]).dropna()

rf1=data_1['Risk-free rate']
rf1=rf1.drop(rf1.index[0:42]).dropna()

atm1=data_1.iloc[:,37] # ATM call Maturity = 45days, Strike = 520, Spot = 516.54
atm1=atm1.drop(atm1.index[0:42]).dropna()

itm1=data_1.iloc[:,31] # ITM  Strike = 490
itm1=itm1.drop(itm1.index[0:42]).dropna()

otm1=data_1.iloc[:,43] # OTM Strike = 550
otm1=otm1.drop(otm1.index[0:42]).dropna()

maturity1=43.0
strike1=520.0


# PRINT HEDGING ERRORS
print("Sheet 1 & 2, Hedging error for portfolio, everyother day: ",
      delta_hedge_portfolio(atm1,spot1,rf1,strike1,maturity1,2,atm2,spot2,rf2)) # everyother day 
print("Sheet 1 & 2, Hedging error for porfolio, every 7 days:    ",
      delta_hedge_portfolio(atm1,spot1,rf1,strike1,maturity1,7,atm2,spot2,rf2)) # once a week 

Sheet 1 & 2, Hedging error for portfolio, everyother day:  70.44442091038377
Sheet 1 & 2, Hedging error for porfolio, every 7 days:     91.2782505442906


#Delta-Vega hedge portfolio of options

In [0]:
def delta_vega_portfolio(call,spot,rf,strike,maturity,frequency,rep_call,rep_spot,rep_rf,rep_maturity,
                         call1,spot1,rf1,strike1,maturity1,rep_call1,rep_spot1,rep_rf1,rep_maturity1):
  n=0 # Days passed
  i=0 # Used for rehedging
    
  # Store errors for calculating MSE

  A=delta_vega(call,spot,rf,strike,maturity,frequency,rep_call,rep_spot,rep_rf,rep_maturity)
  
  B=delta_vega(call1,spot1,rf1,strike1,maturity1,rep_call1,rep_spot1,rep_rf1,rep_maturity1)
  
  return MSE(A)